In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.interpolate import griddata
from scipy.sparse import diags, lil_matrix, csc_matrix
from scipy.sparse.linalg import spsolve


In [3]:

class FSISolver:
    def __init__(self):
        # Domain setup
        self.Lx, self.Ly = 1.0, 1.0
        self.Nx, self.Ny = 100, 100
        self.dx = self.Lx / self.Nx
        self.dy = self.Ly / self.Ny
        
        # Time settings
        self.T = 10.0
        self.dt = 0.01
        self.nsteps = int(self.T / self.dt)
        
        # Physical parameters
        self.Re = 100.0  # Reynolds number
        self.rho_fluid = 1.0  # Fluid density
        self.mu = self.rho_fluid * self.Lx / self.Re  # Fluid viscosity
        self.rho_solid = 1.1  # Solid density
        self.E = 1000.0  # Young's modulus for elastic disc
        self.nu = 0.3  # Poisson's ratio
        
        # Grid setup
        self.x = np.linspace(0, self.Lx, self.Nx+1)
        self.y = np.linspace(0, self.Ly, self.Ny+1)
        self.X, self.Y = np.meshgrid(self.x, self.y)
        
        # Initialize fields
        self.u = np.zeros((self.Ny+1, self.Nx))  # x-velocity
        self.v = np.zeros((self.Ny, self.Nx+1))  # y-velocity
        self.p = np.zeros((self.Ny, self.Nx))    # pressure
        
        # Initialize solid (circular elastic disc)
        self.disc_radius = 0.2
        self.disc_center = np.array([0.6, 0.5])
        self.n_lagrangian = 100  # Number of Lagrangian markers
        self.initialize_solid()
        
        # History for visualization
        self.solid_history = []
        self.velocity_history = []
        
    def initialize_solid(self):
        # Initialize Lagrangian markers for the circular disc boundary
        theta = np.linspace(0, 2*np.pi, self.n_lagrangian, endpoint=False)
        self.X_lag = self.disc_center[0] + self.disc_radius * np.cos(theta)
        self.Y_lag = self.disc_center[1] + self.disc_radius * np.sin(theta)
        self.X_lag_init = self.X_lag.copy()
        self.Y_lag_init = self.Y_lag.copy()
        
        # Interior points for solid (for finite element discretization)
        n_interior = 50  # Number of interior points along radius
        self.X_solid = []
        self.Y_solid = []
        
        # Simple radial distribution of points
        for r in np.linspace(0, self.disc_radius, n_interior+1)[1:]:
            for th in np.linspace(0, 2*np.pi, int(2*np.pi*r/self.disc_radius*50), endpoint=False):
                self.X_solid.append(self.disc_center[0] + r * np.cos(th))
                self.Y_solid.append(self.disc_center[1] + r * np.sin(th))
        
        self.X_solid = np.array(self.X_solid)
        self.Y_solid = np.array(self.Y_solid)
        self.X_solid_init = self.X_solid.copy()
        self.Y_solid_init = self.Y_solid.copy()
    
    def is_in_solid(self, x, y):
        """Check if a point (x,y) is inside the solid."""
        return ((x - self.disc_center[0])**2 + (y - self.disc_center[1])**2) <= self.disc_radius**2
    
    def interpolate_velocity_to_solid(self):
        """Interpolate fluid velocity to Lagrangian markers."""
        # Convert u,v staggered grid to cell centers
        u_center = 0.5 * (self.u[:, :-1] + self.u[:, 1:])
        v_center = 0.5 * (self.v[:-1, :] + self.v[1:, :])
        
        # Points where velocities are defined
        points_u = np.vstack((self.X[:-1, :].flatten(), self.Y[:-1, :].flatten())).T
        points_v = np.vstack((self.X[:, :-1].flatten(), self.Y[:, :-1].flatten())).T
        
        # Interpolate velocities to Lagrangian points
        points_lag = np.vstack((self.X_lag, self.Y_lag)).T
        u_lag = griddata(points_u, u_center.flatten(), points_lag, method='linear', fill_value=0)
        v_lag = griddata(points_v, v_center.flatten(), points_lag, method='linear', fill_value=0)
        
        return u_lag, v_lag
    
    def compute_solid_deformation(self, u_lag, v_lag):
        """Compute solid deformation using a simple elastic model."""
        # Move Lagrangian points based on velocity
        self.X_lag += u_lag * self.dt
        self.Y_lag += v_lag * self.dt
        
        # Compute deformation from initial configuration
        displacement_x = self.X_lag - self.X_lag_init
        displacement_y = self.Y_lag - self.Y_lag_init
        
        # Compute elastic forces (simplified spring model)
        k_spring = self.E / 100  # Spring constant
        force_x = -k_spring * displacement_x
        force_y = -k_spring * displacement_y
        
        # Update interior points based on boundary deformation
        # Using a simple radial basis function interpolation
        points_lag = np.vstack((self.X_lag_init, self.Y_lag_init)).T
        disp_x_interior = griddata(points_lag, displacement_x, 
                               np.vstack((self.X_solid_init, self.Y_solid_init)).T, 
                               method='cubic', fill_value=0)
        disp_y_interior = griddata(points_lag, displacement_y, 
                               np.vstack((self.X_solid_init, self.Y_solid_init)).T, 
                               method='cubic', fill_value=0)
        
        self.X_solid = self.X_solid_init + disp_x_interior
        self.Y_solid = self.Y_solid_init + disp_y_interior
        
        return force_x, force_y
    
    def spread_force_to_fluid(self, force_x, force_y):
        """Spread forces from Lagrangian markers to Eulerian grid."""
        # Initialize force arrays on Eulerian grid
        fx = np.zeros_like(self.u)
        fy = np.zeros_like(self.v)
        
        # Delta function width
        dh = max(self.dx, self.dy)
        delta_width = 2 * dh
        
        # Spread forces using discrete delta function
        for i in range(len(self.X_lag)):
            xl, yl = self.X_lag[i], self.Y_lag[i]
            fx_lag, fy_lag = force_x[i], force_y[i]
            
            # Find nearby Eulerian grid points
            i_min = max(0, int((xl - delta_width) / self.dx))
            i_max = min(self.Nx, int((xl + delta_width) / self.dx) + 1)
            j_min = max(0, int((yl - delta_width) / self.dy))
            j_max = min(self.Ny, int((yl + delta_width) / self.dy) + 1)
            
            # Spread force using a simple discretized delta function
            for j in range(j_min, j_max+1):
                for i in range(i_min, i_max+1):
                    if i < self.Nx and j < self.Ny+1:
                        # Distance to u-grid point
                        x_u = i * self.dx
                        y_u = j * self.dy
                        r_u = np.sqrt((xl - x_u)**2 + (yl - y_u)**2)
                        if r_u < delta_width:
                            # Simple discrete delta function
                            delta_u = max(0, 1 - r_u/delta_width) / (np.pi * delta_width**2)
                            fx[j, i] += fx_lag * delta_u
                    
                    if i < self.Nx+1 and j < self.Ny:
                        # Distance to v-grid point
                        x_v = i * self.dx
                        y_v = j * self.dy
                        r_v = np.sqrt((xl - x_v)**2 + (yl - y_v)**2)
                        if r_v < delta_width:
                            # Simple discrete delta function
                            delta_v = max(0, 1 - r_v/delta_width) / (np.pi * delta_width**2)
                            fy[j, i] += fy_lag * delta_v
        
        return fx, fy
    
    def solve_navier_stokes(self, fx, fy):
        """Solve the Navier-Stokes equations using a projection method."""
        # Create temporary arrays
        u_star = np.zeros_like(self.u)
        v_star = np.zeros_like(self.v)
        
        # Apply boundary conditions
        self.apply_boundary_conditions()
        
        # Step 1: Compute intermediate velocity field (u*)
        for j in range(1, self.Ny):
            for i in range(1, self.Nx-1):
                # x-momentum
                conv_u = (
                    (self.u[j, i+1]**2 - self.u[j, i-1]**2) / (2 * self.dx) +
                    ((self.u[j+1, i] + self.u[j+1, i-1]) * (self.v[j, i] + self.v[j, i+1]) -
                     (self.u[j-1, i] + self.u[j-1, i-1]) * (self.v[j-1, i] + self.v[j-1, i+1])) / (4 * self.dy)
                )
                
                diff_u = (
                    (self.u[j, i+1] - 2*self.u[j, i] + self.u[j, i-1]) / self.dx**2 +
                    (self.u[j+1, i] - 2*self.u[j, i] + self.u[j-1, i]) / self.dy**2
                )
                
                u_star[j, i] = self.u[j, i] + self.dt * (
                    -conv_u + 
                    self.mu/self.rho_fluid * diff_u +
                    fx[j, i]/self.rho_fluid
                )
        
        for j in range(1, self.Ny-1):
            for i in range(1, self.Nx):
                # y-momentum
                conv_v = (
                    ((self.u[j, i] + self.u[j+1, i]) * (self.v[j, i+1] + self.v[j, i]) -
                     (self.u[j, i-1] + self.u[j+1, i-1]) * (self.v[j, i-1] + self.v[j, i])) / (4 * self.dx) +
                    (self.v[j+1, i]**2 - self.v[j-1, i]**2) / (2 * self.dy)
                )
                
                diff_v = (
                    (self.v[j, i+1] - 2*self.v[j, i] + self.v[j, i-1]) / self.dx**2 +
                    (self.v[j+1, i] - 2*self.v[j, i] + self.v[j-1, i]) / self.dy**2
                )
                
                v_star[j, i] = self.v[j, i] + self.dt * (
                    -conv_v + 
                    self.mu/self.rho_fluid * diff_v +
                    fy[j, i]/self.rho_fluid
                )
        
        # Step 2: Solve pressure Poisson equation
        div_u_star = np.zeros((self.Ny, self.Nx))
        for j in range(1, self.Ny-1):
            for i in range(1, self.Nx-1):
                div_u_star[j, i] = (
                    (u_star[j, i] - u_star[j, i-1]) / self.dx +
                    (v_star[j, i] - v_star[j-1, i]) / self.dy
                ) / self.dt
        
        # Construct sparse matrix for Poisson equation
        N = self.Nx * self.Ny
        A = lil_matrix((N, N))
        b = np.zeros(N)
        
        # Fill matrix A and vector b
        for j in range(self.Ny):
            for i in range(self.Nx):
                idx = j * self.Nx + i
                
                if i == 0 or i == self.Nx-1 or j == 0 or j == self.Ny-1:
                    # Boundary conditions (Neumann)
                    A[idx, idx] = 1
                    b[idx] = 0
                else:
                    # Interior points
                    A[idx, idx] = -2/self.dx**2 - 2/self.dy**2
                    A[idx, idx+1] = 1/self.dx**2
                    A[idx, idx-1] = 1/self.dx**2
                    A[idx, idx+self.Nx] = 1/self.dy**2
                    A[idx, idx-self.Nx] = 1/self.dy**2
                    b[idx] = div_u_star[j, i]
        
        # Convert to CSC format for efficient solving
        A = A.tocsc()
        
        # Solve the system
        p_flat = spsolve(A, b)
        p_new = p_flat.reshape((self.Ny, self.Nx))
        
        # Normalize pressure (fix pressure at a reference point)
        p_new = p_new - p_new[0, 0]
        
        # Step 3: Project to get divergence-free velocity
        u_new = np.zeros_like(self.u)
        v_new = np.zeros_like(self.v)
        
        for j in range(1, self.Ny):
            for i in range(1, self.Nx-1):
                u_new[j, i] = u_star[j, i] - self.dt * (p_new[j, i] - p_new[j, i-1]) / self.dx
                
        for j in range(1, self.Ny-1):
            for i in range(1, self.Nx):
                v_new[j, i] = v_star[j, i] - self.dt * (p_new[j, i] - p_new[j-1, i]) / self.dy
        
        # Update fields
        self.u = u_new
        self.v = v_new
        self.p = p_new
        
        # Apply boundary conditions again
        self.apply_boundary_conditions()
    
    def apply_boundary_conditions(self):
        """Apply boundary conditions to velocity field."""
        # Top boundary (moving lid): u=1, v=0
        self.u[-1, :] = 1.0
        
        # Left, right, and bottom boundaries: u=v=0
        self.u[0, :] = 0.0   # Bottom
        self.v[:, 0] = 0.0   # Left
        self.v[:, -1] = 0.0  # Right
        
        # No-slip at solid boundary
        # This is handled through the forcing in the FSI approach
    
    def solve(self):
        """Main solution loop."""
        for step in range(self.nsteps):
            t = step * self.dt
            print(f"Step {step}/{self.nsteps}, t = {t:.3f}")
            
            # Step 1: Interpolate fluid velocity to Lagrangian markers
            u_lag, v_lag = self.interpolate_velocity_to_solid()
            
            # Step 2: Compute solid deformation and resultant forces
            force_x, force_y = self.compute_solid_deformation(u_lag, v_lag)
            
            # Step 3: Spread forces to Eulerian grid
            fx, fy = self.spread_force_to_fluid(force_x, force_y)
            
            # Step 4: Solve Navier-Stokes equations
            self.solve_navier_stokes(fx, fy)
            
            # Store results for visualization
            if step % 10 == 0:
                self.store_results()
        
    def store_results(self):
        """Store current state for visualization."""
        # Store solid configuration
        solid_config = {
            'boundary_x': self.X_lag.copy(),
            'boundary_y': self.Y_lag.copy(),
            'interior_x': self.X_solid.copy(),
            'interior_y': self.Y_solid.copy()
        }
        self.solid_history.append(solid_config)
        
        # Store velocity field (on cell centers for visualization)
        u_center = 0.5 * (self.u[:, :-1] + self.u[:, 1:])
        v_center = 0.5 * (self.v[:-1, :] + self.v[1:, :])
        self.velocity_history.append((u_center.copy(), v_center.copy()))
    
    def visualize_results(self):
        """Visualize the simulation results."""
        # Create figure
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # Function to update plot for animation
        def update(frame):
            ax.clear()
            
            # Plot velocity field as streamplot
            u_vis, v_vis = self.velocity_history[frame]
            x_center = 0.5 * (self.x[:-1] + self.x[1:])
            y_center = 0.5 * (self.y[:-1] + self.y[1:])
            X_center, Y_center = np.meshgrid(x_center, y_center)
            
            # Plot streamlines
            ax.streamplot(X_center, Y_center, u_vis, v_vis, 
                         density=1.5, color='gray', linewidth=0.5, arrowsize=0.5)
            
            # Plot solid boundary
            solid = self.solid_history[frame]
            ax.plot(solid['boundary_x'], solid['boundary_y'], 'b-', linewidth=2)
            ax.scatter(solid['interior_x'], solid['interior_y'], c='lightblue', s=3, alpha=0.5)
            
            # Set plot limits and labels
            ax.set_xlim(0, self.Lx)
            ax.set_ylim(0, self.Ly)
            ax.set_aspect('equal')
            ax.set_title(f'FSI Simulation, t = {frame * 10 * self.dt:.2f}')
            ax.set_xlabel('x')
            ax.set_ylabel('y')
            
            return []
        
        # Create animation
        anim = FuncAnimation(fig, update, frames=len(self.solid_history), 
                            interval=200, blit=True)
        
        # Show plot
        plt.tight_layout()
        plt.show()
        
        return anim


In [5]:
solver = FSISolver()
solver.solve()


Step 0/1000, t = 0.000


ValueError: different number of values and points

In [ ]:

# Create and run solver
anim = solver.visualize_results()

# Save animation (optional)
# anim.save('fsi_simulation.mp4', writer='ffmpeg', fps=10)